In [3]:
!pip install import_ipynb

In [4]:
import import_ipynb
import data_builder
import model_builder
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils.np_utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow 
import pickle 
import numpy as np



importing Jupyter notebook from data_builder.ipynb
     |████████████████████████████████| 2.0MB 3.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


importing Jupyter notebook from model_builder.ipynb


Using TensorFlow backend.


In [5]:

fights_video = "/content/drive/My Drive/code/Peliculas/fights_video"
fights_img = "/content/fights_img"
noFights_img= "/content/noFights_img"
noFights_video="/content/drive/My Drive/code/Peliculas/noFights_video"
data_dir = "/content"
data_builder.save_img(fights_video, fights_img)
data_builder.save_img(noFights_video, noFights_img)

data_dir = "/content"

data_builder.image_builder(data_dir)
 

Successfully created the directory /content/fights_img 
hada howa range len ta3 vid range(0, 188)
vidéo N°  1 ========================================
extract images  frome video number : 1
vidéo N°  2 ========================================
extract images  frome video number : 2
vidéo N°  3 ========================================
extract images  frome video number : 3
vidéo N°  4 ========================================
extract images  frome video number : 4
vidéo N°  5 ========================================
extract images  frome video number : 5
vidéo N°  6 ========================================
extract images  frome video number : 6
vidéo N°  7 ========================================
extract images  frome video number : 7
vidéo N°  8 ========================================
extract images  frome video number : 8
vidéo N°  9 ========================================
extract images  frome video number : 9
vidéo N°  10 ========================================
extract images  from

In [6]:
X = pickle.load(open("X.pickle" , "rb"))
Y = pickle.load(open("Y.pickle" , 'rb'))
y = np.array(Y)

X = X/255.0 
print(len(X))
print(len(y))

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(X, Y,
	test_size=0.25, stratify=Y, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# initialize the optimizer and model
print("[INFO] compiling model...")
epo = 60
opt = Adam(lr=1e-4, decay=1e-4 / epo)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)
print(trainX.shape)

#build the model
model = model_builder.StridedNet.build(width=160, height=160, depth=1,
	classes= len(lb.classes_), reg=l2(0.0005))




16790
16790
[INFO] compiling model...
(12592, 160, 160)


In [7]:
trainX = trainX.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2], 1)
testX = testX.reshape(testX.shape[0], testX.shape[1], testX.shape[2], 1)

print(trainX.shape)
print(testX.shape)

(12592, 160, 160, 1)
(4198, 160, 160, 1)


In [8]:
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 77, 77, 16)        800       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 39, 39, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 39, 39, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 39, 39, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 20, 32)       

In [0]:
# train the network
print("[INFO] training network for {} epochs...".format(epo))
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // 32,
	epochs=60)

[INFO] training network for 60 epochs...
Epoch 1/60
393/393 [==============================] - 133s 337ms/step - loss: 9.5827 - accuracy: 0.3936 - val_loss: 9.4143 - val_accuracy: 0.3935
Epoch 2/60
393/393 [==============================] - 132s 335ms/step - loss: 9.4059 - accuracy: 0.3936 - val_loss: 9.3128 - val_accuracy: 0.3935
Epoch 3/60
393/393 [==============================] - 132s 335ms/step - loss: 9.3336 - accuracy: 0.3940 - val_loss: 9.2756 - val_accuracy: 0.3935
Epoch 4/60
393/393 [==============================] - 132s 335ms/step - loss: 9.3266 - accuracy: 0.3929 - val_loss: 9.2610 - val_accuracy: 0.3935
Epoch 5/60
393/393 [==============================] - 131s 334ms/step - loss: 9.2937 - accuracy: 0.3943 - val_loss: 9.2548 - val_accuracy: 0.3935
Epoch 6/60
393/393 [==============================] - 132s 337ms/step - loss: 9.3150 - accuracy: 0.3929 - val_loss: 9.2519 - val_accuracy: 0.3935
Epoch 7/60
393/393 [==============================] - 133s 338ms/step - loss: 9.287

In [0]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))